In [2]:
instructions = {
    'NOP': ('0','0','0','0','0','0','0','0','0','0','0','0',),
    'HLT': ('0','0','0','0','0','0','0','0','0','0','0','1',),
    'INT': ('0','0','0','0','0','0','0','0','0','0','1','0',),
    'BNZ': ('0','0','0','0','0','0','0','0','0','0','1','1',),
    'BLK': ('0','0','0','0','0','0','0','0','0','1','0','0',),
    'ENB': ('0','0','0','0','0','0','0','0','0','1','0','1',),
    'GLA': ('0','0','0','0','0','0','0','1','0','X','X','X',),
    'GET': ('0','0','0','0','0','0','0','1','1','X','X','X',),
    'LOD': ('0','0','0','0','0','0','1','0','0','X','X','X',),
    'STR': ('0','0','0','0','0','0','1','0','1','X','X','X',),
    'PSH': ('0','0','0','0','0','0','1','1','0','X','X','X',),
    'POP': ('0','0','0','0','0','0','1','1','1','X','X','X',),
    'LDI': ('0','0','0','0','1','X','X','X','X','X','X','X',),
    'LSH': ('0','0','0','1','0','0','X','X','X','X','X','X',),
    'RSH': ('0','0','0','1','0','1','X','X','X','X','X','X',),
    'INC': ('0','0','0','1','1','0','X','X','X','X','X','X',),
    'DEC': ('0','0','0','1','1','1','X','X','X','X','X','X',),
    'AND': ('0','0','1','X','X','X','X','X','X','X','X','X',),
    'OR':  ('0','1','0','X','X','X','X','X','X','X','X','X',),
    'NAD': ('0','1','1','X','X','X','X','X','X','X','X','X',),
    'SUB': ('1','0','0','X','X','X','X','X','X','X','X','X',),
    'XOR': ('1','0','1','X','X','X','X','X','X','X','X','X',),
    'NOR': ('1','1','0','X','X','X','X','X','X','X','X','X',),
    'ADD': ('1','1','1','X','X','X','X','X','X','X','X','X',),
}

In [3]:
controls = {
    'Halt': ['HLT'],
    'Interrupt': ['INT'],
    'Branch if Zero': ['BNZ'],
    'Block interrupts': ['BLK'],
    'Unblock interrupts': ['ENB'],
    'Read Last Address': ['GLA'],
    'Read Interrupt Device': ['GET'],
    'Read Memory': ['LOD','POP'],
    'Write Memory': ['STR','PSH'],
    'Use Memory Pointer': ['LOD','STR'],
    'Use Stack Pointer': ['PSH','POP'],
    'Load Immediate': ['LDI'],
    'Write Reg 1': ['GLA','GET','LOD','PSH','LSH','RSH','INC','DEC','AND','OR','NAD','SUB','XOR','NOR','ADD'],
    'Read Reg 1': ['STR','POP'],
    'Read Reg 2': ['LSH','RSH','INC','DEC','AND','OR','NAD','SUB','XOR','NOR','ADD'],
    'Read Reg 3': ['AND','OR','NAD','SUB','XOR','NOR','ADD'],
    'Left Shift': ['LSH'],
    'Right Shift': ['RSH'],
    'AND': ['AND','NAD'],
    'OR': ['OR','NOR'],
    'XOR': ['XOR'],
    'NOT out': ['NAD','NOR'],
    'NOT Reg 3': ['DEC','SUB'],
    'Add': ['INC','DEC','SUB','ADD'],
    'Inject Carry': ['INC','DEC','SUB'],
    'One as Reg 3': ['DEC'],
}

In [4]:
def equ(control: list[str]) -> str:
    inputs =  ['A','B','C','D','E','F','G','H','I','J','K','L']
    equ = ""

    for instruction in control:
        equ += '('
        last = ''
        for i, bit in enumerate(reversed(instructions[instruction])):
            if i != 0 and last != 'X':
                equ += ' & '

            if bit == '0':
                equ += f"~{inputs[i]}"
            elif bit == '1':
                equ += f"{inputs[i]}"
            elif bit == 'X':
                equ += f"({inputs[i]} | ~{inputs[i]})"
            # last = bit
        equ += ') | '

    return equ[:-3]

In [5]:
equ(controls['Halt'])

'(A & ~B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)'

In [6]:
control_equs = {}

for key, value in controls.items():
    control_equs[key] = equ(value)

control_equs

{'Halt': '(A & ~B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Interrupt': '(~A & B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Branch if Zero': '(A & B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Block interrupts': '(~A & ~B & C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Unblock interrupts': '(A & ~B & C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Read Last Address': '((A | ~A) & (B | ~B) & (C | ~C) & ~D & E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Read Interrupt Device': '((A | ~A) & (B | ~B) & (C | ~C) & D & E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Read Memory': '((A | ~A) & (B | ~B) & (C | ~C) & ~D & ~E & F & ~G & ~H & ~I & ~J & ~K & ~L) | ((A | ~A) & (B | ~B) & (C | ~C) & D & E & F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Write Memory': '((A | ~A) & (B | ~B) & (C | ~C) & D & ~E & F & ~G & ~H & ~I & ~J & ~K & ~L) | ((A | ~A) & (B | ~B) & (C | ~C) & ~D & E & F & ~G & ~H & ~I & ~J & ~K & ~L)',
 'Use Memory Pointer': '((A | ~A) & (B | ~B) & (C | 

In [7]:
for k, v in control_equs.items():
    print(k, '=', v, end='\n\n')

Halt = (A & ~B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)

Interrupt = (~A & B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)

Branch if Zero = (A & B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)

Block interrupts = (~A & ~B & C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)

Unblock interrupts = (A & ~B & C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)

Read Last Address = ((A | ~A) & (B | ~B) & (C | ~C) & ~D & E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)

Read Interrupt Device = ((A | ~A) & (B | ~B) & (C | ~C) & D & E & ~F & ~G & ~H & ~I & ~J & ~K & ~L)

Read Memory = ((A | ~A) & (B | ~B) & (C | ~C) & ~D & ~E & F & ~G & ~H & ~I & ~J & ~K & ~L) | ((A | ~A) & (B | ~B) & (C | ~C) & D & E & F & ~G & ~H & ~I & ~J & ~K & ~L)

Write Memory = ((A | ~A) & (B | ~B) & (C | ~C) & D & ~E & F & ~G & ~H & ~I & ~J & ~K & ~L) | ((A | ~A) & (B | ~B) & (C | ~C) & ~D & E & F & ~G & ~H & ~I & ~J & ~K & ~L)

Use Memory Pointer = ((A | ~A) & (B | ~B) & (C | ~C) & ~D & ~E & F & ~G & ~H & ~I & ~J &

# Simplifier

In [15]:
from sympy import symbols, Not, And, Or, simplify_logic

A, B, C, D, E, F, G, H, I, J, K, L = symbols('A B C D E F G H I J K L')

def equ_s(control: list[str]):
    inputs =  [A, B, C, D, E, F, G, H, I, J, K, L]
    equ = []

    for instruction in control:
        sub_equ = []
        for i, bit in enumerate(reversed(instructions[instruction])):
            if bit == '0':
                sub_equ.append(Not(inputs[i]))
            elif bit == '1':
                sub_equ.append(inputs[i])
        equ.append(And(*sub_equ))

    return Or(*equ)

for key, value in controls.items():
    expr = equ_s(value)
    cnf_expr = simplify_logic(expr, form='cnf', force=True)
    dnf_expr = simplify_logic(expr, form='dnf', force=True)
    print(key, '(ORG) =', expr, end='\n\n')
    print(key, '(CNF) =', cnf_expr, end='\n\n')
    print(key, '(DNF) =', dnf_expr, end='\n\n\n\n')

Halt (ORG) = A & ~B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Halt (CNF) = A & ~B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Halt (DNF) = A & ~B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L



Interrupt (ORG) = B & ~A & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Interrupt (CNF) = B & ~A & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Interrupt (DNF) = B & ~A & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L



Branch if Zero (ORG) = A & B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Branch if Zero (CNF) = A & B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Branch if Zero (DNF) = A & B & ~C & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L



Block interrupts (ORG) = C & ~A & ~B & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Block interrupts (CNF) = C & ~A & ~B & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L

Block interrupts (DNF) = C & ~A & ~B & ~D & ~E & ~F & ~G & ~H & ~I & ~J & ~K & ~L



Unblock interrupts (ORG) = A & C & ~B & ~D & ~E & ~F & ~G & ~H